In [1]:
## 필요한 라이브러리를 불러옵니다. 
import time
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from flask import Flask, send_file
import redis

## pyspark를 불러옵니다.
from pyspark import SparkContext

## 이미 spark context가 살아 있으면 죽이고, 있으면 그대로 사용합니다. 
## spark context는 한 번에 여러 개 돌리려면 세팅에 몇 개를 추가해야 하는것 같아요. 
## 또한 스파크 컨텍스트를 제대로 구현하려면 여기에 넘겨야 하는 값들이 많이 있습니다만, 저는 테스트만 하려고 해서 이걸 대충 세팅했습니다. 


In [2]:
# from flask import Flask, send_file

# from io import BytesIO, StringIO
# import numpy as np 

# ## macOS의 경우 아래 순서에 따라서 library를 import해줘야 에러없이 잘 됩니다. 
# import matplotlib
# matplotlib.use('Agg')
# import matplotlib.pyplot as plt
# #################

# app = Flask(__name__, static_url_path='/static')

# ## mean, var를 url에서 입력받아서 적합한 그림 파일을 만들어준다. 
# ## url주소와 함수 이름은 같아야 함
# ## url의 argument와 함수의 argument는 같아야 함
# @app.route('/fig/<int:mean>_<int:var>')
# def fig(mean, var):
#   plt.figure(figsize=(4, 3))
#   ## url에서 입력받은 mean, var를 그대로 사용하여 random sampling
#   xs = np.random.normal(mean, var, 100)
#   ys = np.random.normal(mean, var, 100)
#   plt.scatter(xs, ys, s=100, marker='h', color='red', alpha=0.3)
#   ## file로 저장하는 것이 아니라 binary object에 저장해서 그대로 file을 넘겨준다고 생각하면 됨
#   ## binary object에 값을 저장한다. 
#   ## svg로 저장할 수도 있으나, 이 경우 html에서 다른 방식으로 저장해줘야 하기 때문에 일단은 png로 저장해줌
#   img = BytesIO()
#   plt.savefig(img, format='png', dpi=200)
#   ## object를 읽었기 때문에 처음으로 돌아가줌
#   img.seek(0)
#   return send_file(img, mimetype='image/png')
  
#   # plt.savefig(img, format='svg')
#   # return send_file(img, mimetype='image/svg')

# if __name__ == '__main__':
#   app.run(debug=True)

In [3]:
sc = SparkContext(master="local", appName="first app")

In [4]:
app = Flask(__name__, static_url_path='/static')

In [5]:
@app.route('/')
def fig():
    if sc is None: 
        sc = SparkContext(master="local", appName="first app")
    else:
        sc.stop()
        sc = SparkContext(master="local", appName="first app")

## 이미 spark context가 살아 있으면 죽이고, 있으면 그대로 사용합니다. 
## spark context는 한 번에 여러 개 돌리려면 세팅에 몇 개를 추가해야 하는것 같아요. 
## 또한 스파크 컨텍스트를 제대로 구현하려면 여기에 넘겨야 하는 값들이 많이 있습니다만, 저는 테스트만 하려고 해서 이걸 대충 세팅했습니다. 
    if sc is None: 
        sc = SparkContext(master="local", appName="first app")
    else:
        sc.stop()
        sc = SparkContext(master="local", appName="first app")

    ### peformance check 
    spark_time_lst = []
    python_time_lst = []
    python_np_time_lst = []
    n_lst = [20000*i for i in range(0, 10)]
    for n in n_lst:
        def each_k(k):
            return 1/(16**k)*( 4/(8*k+1) - 2/(8*k+4) - 1/(8*k+5) - 1/(8*k+6))
        ## with spark 
        start_time = time.time()
        pi_approximated = sc.parallelize(range(0, n)).map(each_k).sum()
        spark_time_lst.append(time.time()-start_time)
        ## pure python
        start_time = time.time()
        pi_approximated = sum((each_k(k) for k in range(0, n)))
        python_time_lst.append(time.time()-start_time)
        ## with numpy 
        start_time = time.time()
        pi_approximated = np.apply_along_axis(arr=np.array(range(0, 10)), func1d=each_k, axis=0).sum()
        python_np_time_lst.append(time.time() - start_time)

    ### plotting     
    df = pd.DataFrame({
        "spark":spark_time_lst.copy(), 
        "pure python":python_time_lst.copy(), 
        "python with numpy":python_np_time_lst.copy()
    }, index = [20000*i for i in range(0, 10)])
    plt.figure(figsize=(12, 6))
    plt.plot(df['spark'], 'ro-'), plt.plot(df['pure python'], 'bo-'), plt.plot(df['python with numpy'], 'go-')
    plt.legend(fontsize=25)
    plt.xticks([20000*i for i in range(0, 10)], [20000*i for i in range(0, 10)])
    img = ByteIO()
    plt.savefig(img, format='png',dpi =200)
    img.seek(0)
    return send_file(img, mimetype='image/png')
   # plt.show()
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 1

In [ ]:

if sc is None: 
    sc = SparkContext(master="local", appName="first app")
else:
    sc.stop()
    sc = SparkContext(master="local", appName="first app")

## 이미 spark context가 살아 있으면 죽이고, 있으면 그대로 사용합니다. 
## spark context는 한 번에 여러 개 돌리려면 세팅에 몇 개를 추가해야 하는것 같아요. 
## 또한 스파크 컨텍스트를 제대로 구현하려면 여기에 넘겨야 하는 값들이 많이 있습니다만, 저는 테스트만 하려고 해서 이걸 대충 세팅했습니다. 
if sc is None: 
    sc = SparkContext(master="local", appName="first app")
else:
    sc.stop()
    sc = SparkContext(master="local", appName="first app")


In [ ]:

### peformance check 
spark_time_lst = []
python_time_lst = []
python_np_time_lst = []
n_lst = [20000*i for i in range(0, 10)]


In [ ]:
for n in n_lst:
    def each_k(k):
        return 1/(16**k)*( 4/(8*k+1) - 2/(8*k+4) - 1/(8*k+5) - 1/(8*k+6))
    ## with spark 
    start_time = time.time()
    pi_approximated = sc.parallelize(range(0, n)).map(each_k).sum()
    spark_time_lst.append(time.time()-start_time)
    ## pure python
    start_time = time.time()
    pi_approximated = sum((each_k(k) for k in range(0, n)))
    python_time_lst.append(time.time()-start_time)
    ## with numpy 
    start_time = time.time()
    pi_approximated = np.apply_along_axis(arr=np.array(range(0, 10)), func1d=each_k, axis=0).sum()
    python_np_time_lst.append(time.time() - start_time)


In [ ]:

### plotting     
df = pd.DataFrame({
    "spark":spark_time_lst.copy(), 
    "pure python":python_time_lst.copy(), 
    "python with numpy":python_np_time_lst.copy()
}, index = [20000*i for i in range(0, 10)])
plt.figure(figsize=(12, 6))
plt.plot(df['spark'], 'ro-'), plt.plot(df['pure python'], 'bo-'), plt.plot(df['python with numpy'], 'go-')
plt.legend(fontsize=25)
plt.xticks([20000*i for i in range(0, 10)], [20000*i for i in range(0, 10)])
plt.savefig('/home/180831_pyspark_performance_check.svg')
plt.show()

In [ ]:
!pwd